In [ ]:
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
import time
import re
import pandas as pd

In [ ]:
URL_obs = 'http://www.hko.gov.hk/en/cis/dailyExtract.htm?y=2000&m=1'
URL_year = re.search(r'y=(\d{4})',URL_obs).group(1)
URL_month = re.search(r'\&m=(\d+)$',URL_obs).group(1)
#Please change the path to the location of your chromedriver
driver = webdriver.Chrome('/Users/Hei/Applications/chromedriver')

In [ ]:
#Initialize dict for storing table data
dict_wthr_table = {'Year':[],'Month':[],'Day':[],'Mean Pressure(hPa)':[],'Absolute Daily Max(deg. C)':[],'Mean(deg. C)':[],\
                 'Absolute Daily min(deg. C)':[],'Mean Dew Point(deg. C)':[],'Mean Relative Humidity(%)':[],\
                 'Mean Amount of Cloud(%)':[],'Total Rainfall(mm)':[],'Total Bright Sunshine(hours)':[],\
                 'Prevailling Wind Direction(degrees)':[],'Mean Wind Speed(km/h)':[]}


# while int(URL_year)<2020 or int(URL_month)<10:
while int(URL_year)<2020:
    
    #Get html data by beautiful soup
    driver.get(URL_obs)
    time.sleep(6)
    subhtml = driver.page_source
    soup = BeautifulSoup(subhtml, 'html.parser')

    #Get date from html
    date = soup.find_all(class_="td1_year_class")
    date=[d.get_text() for d in date[:-2]]
    #Get weather detail from html
    weather = soup.find_all(class_="td1_normal_class")
    
    print(URL_month)
    for d in date:
        current_day = int(d)
        start_index=(current_day-1)*11
        end_index=(current_day)*11
        line=[data.get_text() for data in weather[start_index:end_index]]
        dict_wthr_table['Year'].append(URL_year)
        dict_wthr_table['Month'].append(URL_month)
        dict_wthr_table['Day'].append(d)
        print(d,line)
        for i in range(len(line)):
            if i%11==0:
                dict_wthr_table['Mean Pressure(hPa)'].append(line[i])
            elif i%11==1:
                dict_wthr_table['Absolute Daily Max(deg. C)'].append(line[i])
            elif i%11==2:
                dict_wthr_table['Mean(deg. C)'].append(line[i])
            elif i%11==3:
                dict_wthr_table['Absolute Daily min(deg. C)'].append(line[i])
            elif i%11==4:
                dict_wthr_table['Mean Dew Point(deg. C)'].append(line[i])
            elif i%11==5:
                dict_wthr_table['Mean Relative Humidity(%)'].append(line[i])
            elif i%11==6:
                dict_wthr_table['Mean Amount of Cloud(%)'].append(line[i])
            elif i%11==7:
                dict_wthr_table['Total Rainfall(mm)'].append(line[i])
            elif i%11==8:
                dict_wthr_table['Total Bright Sunshine(hours)'].append(line[i])
            elif i%11==9:
                dict_wthr_table['Prevailling Wind Direction(degrees)'].append(line[i])
            elif i%11==10:
                dict_wthr_table['Mean Wind Speed(km/h)'].append(line[i])
    
    #update URL
    if URL_month=='12':
        URL_month='1'
        URL_year=str(int(URL_year)+1)
    else:
        URL_month= str(int(URL_month)+1)
    URL_obs='http://www.hko.gov.hk/en/cis/dailyExtract.htm?y='+URL_year+'&m='+URL_month

In [ ]:
df_wthr_table=pd.DataFrame(dict_wthr_table)
df_wthr_table.to_csv('weather_hk.csv',index=False)

In [ ]:
df_wthr_table.shape

In [ ]:
df_wthr_table